# Setup

 - [OpenAI](https://beta.openai.com/)
 - [python-dotenv](https://pypi.org/project/python-dotenv/)

        python -m venv venv
        source venv/bin/activate
        pip install openai python-dotenv
        pip freeze >> requirements.txt

 - **.env**

     - [OpenAI Key](https://beta.openai.com/account/api-keys)

            OPENAI_API_KEY=